# Car Rental 1

## Objective and Prerequisites

This model is an example of a Car Rental problem. A car rental enterprise wants to determine how many cars to own and where should they be located at the start of each day of a week to maximize weekly profits. This problem is formulated as a linear programming problem using the Gurobi Python API and solved with the Gurobi Optimizer.

This model is example 25 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 284-286 and 340-342.

This example is at the intermediate level, where we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MUI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_CAR_RENTAL1_COM_EVAL_GitHub&utm_term=Car_Rental1&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_CAR_RENTAL1_ACADEMIC_EVAL_GitHub&utm_term=Car_Rental1&utm_content=C_JPM) as an *academic user*.

---
## Problem Description

A car rental enterprise (which rents only one type of car) has depots in Glasgow, Manchester, Birmingham, and Plymouth. We estimate the demand for each day of the week except Sunday (when the enterprise is closed). The following table shows these estimates. We assume that it is not necessary to meet all the demand.


![weeklyDemand](weeklyDemand.PNG)

There are three options to rent a car: one, two, or three days rental. A customer can return the vehicle at the start of the next morning to either the depot from which it was rented or another depot. For example, let’s consider a customer that rents a vehicle for two days on a Thursday, then the vehicle has to be returned on Saturday morning. If a customer rents a vehicle on Friday for three days, the vehicle has to be returned on Tuesday morning. Another example is that the customer rents the vehicle for one day on Saturday. In this case, the vehicle has to be returned on Monday morning;  or if the vehicle is rented for two days, the vehicle has to be returned on Tuesday morning.

The time period to rent a car is independent of the origin and destination. We know from historical data that the distribution of rental periods is as follows:
* 55% of the vehicles are rented for one day,
* 20% for two days,
* and 25% for three days.

The following table shows the estimates of percentages of vehicles rented from each depot and returned to a given depot (independent of the day).

![FromToPct](FromToPct.PNG)

The following table shows the marginal cost of renting a vehicle.

| Days rented | Marginal cost |
| --- | --- |
| 1-day | $\$ 20$ |
| 2-day | $\$ 25$ |
| 3-day | $\$ 30$ |

The ‘opportunity cost’  of owning a vehicle is $\$ 15$ per week.

Undamaged vehicles can be transferred from one depot to another depot, independently of the distance. A customer cannot rent out a vehicle on the day in which this vehicle is transferred. The following table shows the cost of the transfer.

![FromToCst](FromToCst.PNG)

The percentage of returned cars that are damaged is 10%. In this case, the customer needs to pay an extra $100. The damaged vehicle has to be transferred to a repair depot, where it will be repaired the following day. We assume that the cost of transferring a damaged vehicle is the same as transferring an undamaged one. Notice that when the repair depot is the current depot, there is no transfer cost. It takes one day to transfer a damaged vehicle unless it is already at a repair depot. It takes one day to repair a vehicle at the repair depot. There are only two depots with repair capacity. The following table shows the capacity (cars/day) of each repair vehicle.

| Repair depot | Capacity |
| --- | --- |
| Manchester | 12 |
| Birmingham | 20 |

A repaired vehicle is available for rent at the depot the next day or may be transferred to another depot (taking a day). For example, a vehicle that is returned damaged on a Tuesday morning is transferred to a repair depot (if not the current depot) on Tuesday, repaired on Wednesday, and is available for rent at the repair depot on Thursday morning. The price to rent a vehicle depends on the number of days for which the car is rented and whether it is returned to the same depot or not. The following table shows the rental prices (in USD).

![RentalPrice](RentalPrice.PNG)

We make the following assumptions at the beginning of each day:
* Clients return vehicles that are due that day.
* Rental enterprise sends damaged vehicles to the repair depot.
* Transferred vehicles from other depots arrive.
* Rental enterprise sends out transfer vehicles.
* Customers rent out vehicles.
* Repaired vehicles at a repair depot are available for rental.

The goal is to determine the number of vehicles the car rental enterprise should own and where they should be located at the start of each weekday to maximize weekly profit. The enterprise wants a stable solution in which the same expected number of vehicles will be located at the same depot on the same day of subsequent weeks.

---
## Model Formulation

$d,d2 \in \text{Depots}=\{\text{Glasgow}, \text{Manchester}, \text{Birmingham},  \text{Plymouth}\}$

$\text{NRD}=\{\text{Glasgow}, \text{Plymouth}\}$: Depots without repair capacity.

$\text{RD}=\{\text{Manchester}, \text{Birmingham}\}$: Depots with repair capacity.

$t \in \text{Days}=\{\text{Monday},\text{Tuesday},\text{Wednesday},\text{Thursday},\text{Friday},\text{Saturday}\}$

$r \in \text{RentDays}=\{1,2,3\}$: Number of days rented.

### Parameters

$\text{demand}_{d,t} \in \mathbb{R}^+$: Estimated rental demand at depot $d$ on day $t$.

$\text{pctDepot}_{d,d2} \in \mathbb{R}^+$: Proportion of cars rented at depot $d$ to be returned to depot $d2$.

$\text{cstTransfer}_{d,d2} \in \mathbb{R}^+$: Transfer cost of a car from depot $d$ to depot $d2$.

$\text{pctRent}_{r} \in \mathbb{R}^+$: Proportion of cars rented for $r$ days.

$\text{capRepair}_{d} \in \mathbb{R}^+$: Repair capacity of depot $d$.

$\text{cstSameDepot}_{r} \in \mathbb{R}^+$: Rental cost for $r$ days with return to same depot.

$\text{cstOtherDepot}_{r} \in \mathbb{R}^+$: Rental cost for $r$ days with return to other depot.

$\text{marginalCost}_{r} \in \mathbb{R}^+$: Marginal cost to company of  $r$ days rental of a car.

$\text{pctUndamaged } \in [0,1]$: Percent of cars returned by customers that are undamaged.

$\text{pctDamaged  } \in [0,1]$: Percent of cars returned by customers that are damaged.

$\text{cstOwn} \in \mathbb{R}^+$: Cost of owning a car.

$\text{damagedFee} = 10$: Damaged car fee. Ten percent of the cars are damaged and the fee for a damaged car is $\$100$.

### Decision Variables

$\text{xOwned} \in \mathbb{R}^+$: Total number of cars owned.

$\text{xUndamaged}_{d,t} \in \mathbb{R}^+$: Number of undamaged cars available at depot $d$ at the beginning of day $t$.

$\text{xDamaged}_{d,t} \in \mathbb{R}^+$: Number of damaged cars available at depot $d$ at the beginning of day $t$.

$\text{xRented}_{d,t} \in \mathbb{R}^+$: number of cars rented  out from depot $d$ at the beginning of day $t$.

$\text{xUDleft}_{d,t} \in \mathbb{R}^+$: Number of undamaged cars available at depot $d$ at the beginning of day $t$.

$\text{xDleft}_{d,t} \in \mathbb{R}^+$: Number of damaged cars left in depo $d$ at the end of day  $t$.

$\text{xUDtransfer}_{d,d2,t} \in \mathbb{R}^+$: Number of undamaged cars  at depot $d$ at the beginning of day $t$, to be transferred to depot $d2$. 

$\text{xDtransfer}_{d,d2,t} \in \mathbb{R}^+$: Number of damaged cars at depot $d$ at the beginning of day $t$, to be transferred to depot $d2$. 

$\text{xRepaired}_{d,t} \in \mathbb{R}^+$: Number of damaged cars to be repaired at depot $d$ during day $t$.

### Objective function
The objective is to maximize profit.

\begin{equation}
\sum_{d \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d}*\text{pctRent}_{r}*(\text{cstSameDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
+ \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d2}*\text{pctRent}_{r}*(\text{cstOtherDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
- \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}} \text{cstTransfer}_{d,d2}*(\text{xUDtransfer}_{d,d2,t} + \text{xDtransfer}_{d,d2,t} )
- \text{cstOwn}*\text{xOwned}
\end{equation}


### Constraints

**Undamaged cars at a non-repair depot** <br />
Number of undamaged cars available at a non-repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

Demand of undamaged cars at the non-repair depot $d$ during day $t$.

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**Undamaged cars at a repair depot** <br />
Number of undamaged cars available at a repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xRepaired}_{d,(t-1)mod(6)}  + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

Demand of undamaged cars at the repair depot $d$ during day $t$.

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**Damaged cars at a non-repair depot** <br />
Number of damaged cars available at a non-repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xDamaged}_{d,t} \quad \forall d \in NRD, t \in Days
\end{equation}

Demand of undamaged cars at the non-repair depot $d$ during day $t$.

\begin{equation}
\text{xDamaged}_{d,t} = 
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**Damaged cars at a repair depot** <br />
Number of damaged cars available at a repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xdamaged}_{d,t} 
\quad \forall d \in RD, t \in Days
\end{equation}

Demand of undamaged cars at the non-repair depot $d$ during day $t$.

\begin{equation}
\text{xDamaged}_{d,t} = \text{xRepaired}_{d,t} +
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in RD, t \in Days
\end{equation}

**Depot Capacity** <br />
Repair capacity of depot $d$ for each day $t$. 

\begin{equation}
\text{xRepaired}_{d,t} \leq \text{capRepair}_{d}
\quad \forall d \in Depots, t \in Days
\end{equation}

**Depot Demand** <br />
Demand at depot $d$ for each day $t$. 

\begin{equation}
\text{xRented}_{d,t} \leq \text{demand}_{d,t}
\quad \forall d \in Depots, t \in Days
\end{equation}

**Number of cars** <br />
This constraint entails calculating the total number of vehicles that the enterprise needs to own. The total number of vehicles owned should be equal to the number of vehicles rented out from all depots on Monday for three days, plus those on Tuesday for 2 or 3 days, plus all damaged and undamaged cars in depots at the beginning of Wednesday. The rationale behind this equation follows. Let’s pick a day (Wednesday), count the cars undamaged and damaged that were returned to the depots and that are available on Wednesday morning. Let’s count the vehicles that have been rented and have not been returned: Cars rented on Monday for three days, and cars rented on Tuesday for 2 or 3 days.

\begin{equation}
\sum_{d \in \text{Depots}} (0.25*\text{xRented}_{d,0} + 0.45*\text{xRented}_{d,1} + \text{xUndamaged}_{d,2}  + \text{xdamaged}_{d,2} ) = \text{xOwned}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [1]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

## Input data

We define all the input data for the model.

In [2]:
# list of depots and working days of a week

depots = ['Glasgow','Manchester','Birmingham','Plymouth']
NRD = ['Glasgow','Plymouth'] # Non-repair depot
RD =['Manchester','Birmingham'] # Repair depot

days = [0,1,2,3,4,5] # Monday = 0, Tuesday = 1, ...  Saturday = 5
rentDays = [1,2,3]

d2w, demand = gp.multidict({
    ('Glasgow',0): 100,
    ('Glasgow',1): 150,
    ('Glasgow',2): 135,
    ('Glasgow',3): 83,
    ('Glasgow',4): 120,
    ('Glasgow',5): 230,
    ('Manchester',0): 250,
    ('Manchester',1): 143,
    ('Manchester',2): 80,
    ('Manchester',3): 225,
    ('Manchester',4): 210,
    ('Manchester',5): 98,
    ('Birmingham',0): 95,
    ('Birmingham',1): 195,
    ('Birmingham',2): 242,
    ('Birmingham',3): 111,
    ('Birmingham',4): 70,
    ('Birmingham',5): 124,
    ('Plymouth',0): 160,
    ('Plymouth',1): 99,
    ('Plymouth',2): 55,
    ('Plymouth',3): 96,
    ('Plymouth',4): 115,
    ('Plymouth',5): 80
})

#repairCap
depots, capacity = gp.multidict({
    ('Glasgow'): 0,
    ('Manchester'): 12,
    ('Birmingham'): 20,
    ('Plymouth'): 0
})

# Create a dictionary to capture 
# pctRent: percentage of cars rented for r days 
# cstMarginal: marginal cost for renting a car for r days
# prcSameD: price of renting a car r days and returning to same depot
# prcOtherD: price of renting a car r days and returning to another depot
rentDays, pctRent, costMarginal, priceSameD, priceOtherD = gp.multidict({
    (1): [0.55,20,50,70],
    (2): [0.20,25,70,100],
    (3): [0.25,30,120,150]
})

# Cost of owing a car per week.
cstOwn = 15

# Proportional damaged car fee
damagedFee = 10

# Create a dictionary to capture the proportion of cars rented at depot d to be returned to depot d2 
d2d, pctFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.6,
    ('Glasgow','Manchester'): 0.2,
    ('Glasgow','Birmingham'): 0.1,
    ('Glasgow','Plymouth'): 0.1,
    ('Manchester','Glasgow'): 0.15,
    ('Manchester','Manchester'): 0.55,
    ('Manchester','Birmingham'): 0.25,
    ('Manchester','Plymouth'): 0.05,
    ('Birmingham','Glasgow'): 0.15,
    ('Birmingham','Manchester'): 0.2,
    ('Birmingham','Birmingham'): 0.54,
    ('Birmingham','Plymouth'): 0.11,
    ('Plymouth','Glasgow'): 0.08,
    ('Plymouth','Manchester'): 0.12,
    ('Plymouth','Birmingham'): 0.27,
    ('Plymouth','Plymouth'): 0.53
})

# Create a dictionary to capture the transfer costs  of cars
d2d, cstFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.001,
    ('Glasgow','Manchester'): 20,
    ('Glasgow','Birmingham'): 30,
    ('Glasgow','Plymouth'): 50,
    ('Manchester','Glasgow'): 20,
    ('Manchester','Manchester'): 0.001,
    ('Manchester','Birmingham'): 15,
    ('Manchester','Plymouth'): 35,
    ('Birmingham','Glasgow'): 30,
    ('Birmingham','Manchester'): 15,
    ('Birmingham','Birmingham'): 0.001,
    ('Birmingham','Plymouth'): 25,
    ('Plymouth','Glasgow'): 50,
    ('Plymouth','Manchester'): 35,
    ('Plymouth','Birmingham'): 25,
    ('Plymouth','Plymouth'): 0.001
})

# Proportion of undamaged and damaged cars returned
pctUndamaged = 0.9
pctDamaged = 0.1


### Preprocessing
We prepare the data structures to build the linear programming model.

In [3]:
# Build a list of tuples (depot, depot2) such that d != d2
list_d2notd = []

for d,d2 in d2d:
    if (d != d2):
        tp = d,d2
        list_d2notd.append(tp)

d2notd = gp.tuplelist(list_d2notd)

# Build a list of tuples (depot, depot2, day)
list_dd2t = []

for d,d2 in d2notd:
    for t in days:
        tp = d,d2,t 
        list_dd2t.append(tp)
                    
dd2t = gp.tuplelist(list_dd2t)

# Build a list of tuples (depot, rent_day)
list_dr = []

for d in depots:
    for r in rentDays:
        tp = d,r
        list_dr.append(tp)
        
dr = gp.tuplelist(list_dr) 

# Build a list of tuples (depot, day, rent_days )
list_dtr = []

for d in depots:
    for t in days:
            for r in rentDays:
                tp = d,t,r
                list_dtr.append(tp)
                
dtr = gp.tuplelist(list_dtr) 

# Build a list of tuples (depot, depot2, day, rent_days)
list_dd2tr = []

for d,d2 in d2notd:
    for t in days:
        for r in rentDays:
            tp = d,d2,t,r
            list_dd2tr.append(tp)
                    
                    
dd2tr = gp.tuplelist(list_dd2tr)

## Model Deployment
We create a model and the variables. The main decision variables are the number of cars to own
and where should they be located at the start of each day of a week to maximize weekly profits.

In [4]:
model = gp.Model('RentalCar1')

# Number of cars owned
n = model.addVar(name="cars")

# Number of undamaged cars
nu = model.addVars(d2w, name="UDcars")

# Number of damaged cars
nd = model.addVars(d2w, name="Dcars")

# Number of cars hired (rented) cannot exceed their demand
tr = model.addVars(d2w, ub=demand, name="Hcars")
#for d,t in d2w:
    #tr[d,t].lb = 1

# End inventory of undamaged cars
eu = model.addVars(d2w, name="EUDcars")

# End inventory of damaged cars
ed = model.addVars(d2w, name="EDcars")

# Number of undamaged cars transferred
tu = model.addVars(dd2t, name="TUDcars")

# Number of damaged cars transferred
td = model.addVars(dd2t, name="TDcars")

# Number of damaged cars repaired
rp = model.addVars(d2w, name="RPcars")

# Number of damaged cars repaired cannot exceed depot capacity
for d,t in d2w:
    rp[d,t].ub = capacity[d] #repair capacity

Using license file c:\gurobi\gurobi.lic


### Constraints
The number of undamaged cars available at a non-repair depot d at the beginning of  day t should be equal to the demand of undamaged cars at the non-repair depot d during day t.

In [5]:
# Undamaged cars into a non-repair depot constraints (left hand side of balance equation -availability)

UDcarsNRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) 
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in NRD for t in days ), 
                             name="UDcarsNRD_L")

# Undamaged cars out of a non-repair depot constraints (right hand side of balance equation -requirements)

UDcarsNRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t )) 
                                + eu[d,t] == nu[d,t] for d in NRD for t in days ), name='UDcarsNRD_R' )

The number of undamaged cars available at a repair depot d at the beginning of  day t should be equal to the demand of undamaged cars at the repair depot d during day t.

In [6]:
# Undamaged cars into a repair depot constraints (left hand side of balance equation -availability)

UDcarsRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) + rp[d, (t-1)%6 ]
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in RD for t in days ), 
                             name="UDcarsRD_L")

# Undamaged cars out of a repair depot constraints (right hand side of balance equation -requirements)

UDcarsRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t ) ) 
                                + eu[d,t] == nu[d,t] for d in RD for t in days ), name='UDcarsRD_R' )

The number of damaged cars available at a non-repair depot d at the beginning of  day t should be equal to the demand of damaged cars at the non-repair depot d during day t.

In [7]:
# Damaged cars into a non-repair depot constraints (left hand side of balance equation -availability)

DcarsNRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in NRD for t in days ), 
                             name="DcarsNRD_L")

# Damaged cars out of a non-repair depot constraints (right hand side of balance equation -requirements)

DcarsNRD_R = model.addConstrs(( gp.quicksum(td[d,d2,t] for d2 in RD ) 
                                + ed[d,t] == nd[d,t] for d in NRD for t in days ), name='DcarsNRD_R' )

The number of damaged cars available at a repair depot d at the beginning of  day t should be equal to the demand of damaged cars at the repair depot d during day t.

In [8]:
# Damaged cars into a repair depot constraints (left hand side of balance equation -availability)

DcarsRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr )
                              + gp.quicksum(td[d2,d,(t-1)%6 ] for d2, dd in d2notd if (dd == d)) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in RD for t in days ), 
                             name="DcarsRD_L")

# Damaged cars out of a repair depot constraints (right hand side of balance equation -requirements)

DcarsND_R = model.addConstrs((rp[d,t] + gp.quicksum(td[d,d2,t ] for d2 in NRD ) 
                                + ed[d,t] == nd[d,t] for d in RD for t in days ), name='DcarsND_R' )

Total number of cars equals the number of cars rented out from all depots on Monday for 3 days, plus those on Tuesday for 2 or 3 days, plus all damaged and undamaged cars in depots at the beginning of Wednesday.

In [9]:
# Total number of cars owned constraint
# Note: 25% of cars are rented for 3 days, and 20% + 25% = 45% of the cars are rented for 2-days or 3-days

carsConstr = model.addConstr((gp.quicksum(0.25*tr[d,0] + 0.45*tr[d,1] + nu[d,2] + nd[d,2] for d in depots ) 
                              == n ),name='carsConstr')

The objective function is to maximize profit.

In [10]:
# Maximize profit objective function

model.setObjective((
    gp.quicksum(pctFromToD[d,d]*pctRent[r]*(priceSameD[r] - costMarginal[r] + damagedFee)*tr[d,t] for d,t,r in dtr )
    + gp.quicksum(pctFromToD[d,d2]*pctRent[r]*(priceOtherD[r]-costMarginal[r]+damagedFee)*tr[d,t] for d,d2,t,r in dd2tr)
    - gp.quicksum(cstFromToD[d,d2]*tu[d,d2,t] for d,d2,t in dd2t) 
    - gp.quicksum(cstFromToD[d,d2]*td[d,d2,t] for d,d2,t in dd2t) - cstOwn*n ), GRB.MAXIMIZE)

In [11]:
# Verify model formulation

model.write('CarRental1.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97 rows, 289 columns and 1061 nonzeros
Model fingerprint: 0x7ddb81fe
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+01, 3e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 49 rows and 85 columns
Presolve time: 0.01s
Presolved: 48 rows, 204 columns, 936 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1102412e+05   3.626166e+02   0.000000e+00      0s
      69    1.2116021e+05   0.000000e+00   0.000000e+00      0s

Solved in 69 iterations and 0.01 seconds
Optimal objective  1.211602072e+05


---
## Analysis

In [12]:
# Output report

# Total number of cars owned
print(f"The optimal number of cars to be owned is: {round(n.x)}.")

# Optimal profit
print(f"The optimal profit is: {'${:,.2f}'.format(round(model.objVal,2))}.")


The optimal number of cars to be owned is: 617.
The optimal profit is: $121,160.21.


In [13]:
# Create a list to translate the number label of each day to the actual name of the day
dayname = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

# Number of undamaged cars in depot at the beginning of each day.
print("\n\n_________________________________________________________________________________")
print(f"Estimated number of undamaged cars in depot at the beginning of each day: ")
print("_________________________________________________________________________________")

undamaged_cars = pd.DataFrame(columns=['Day','Glasgow','Manchester','Birmingham','Plymouth'])
for t in days:
    undamaged_cars = undamaged_cars.append({"Day": dayname[t], "Glasgow": round(nu['Glasgow',t].x), "Manchester":  round(nu['Manchester',t].x), 'Birmingham':  round(nu['Birmingham',t].x),'Plymouth': round(nu['Plymouth',t].x)   }, ignore_index=True) 
undamaged_cars.index=[''] * len(undamaged_cars)
undamaged_cars



_________________________________________________________________________________
Estimated number of undamaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,68,98,146,41
,Tuesday,66,95,155,40
,Wednesday,70,100,123,43
,Thursday,68,114,116,42
,Friday,70,102,124,43
,Saturday,67,95,158,40


In [14]:
# Number of Damaged cars in depot at the beginning of each day.
print("_________________________________________________________________________________")
print(f"Estimated number of damaged cars in depot at the beginning of each day: ")
print("_________________________________________________________________________________")

damaged_cars = pd.DataFrame(columns=['Day','Glasgow','Manchester','Birmingham','Plymouth'])
for t in days:
    damaged_cars = damaged_cars.append({"Day": dayname[t], "Glasgow": round(nd['Glasgow',t].x), "Manchester":  round(nd['Manchester',t].x), 'Birmingham':  round(nd['Birmingham',t].x),'Plymouth': round(nd['Plymouth',t].x)   }, ignore_index=True) 
damaged_cars.index=[''] * len(damaged_cars)
damaged_cars

_________________________________________________________________________________
Estimated number of damaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,8,12,20,6
,Tuesday,7,12,20,4
,Wednesday,8,13,20,5
,Thursday,8,12,21,5
,Friday,8,12,20,7
,Saturday,7,12,22,4


In [15]:
# Undamaged car rented out from each depot and day.
print("_________________________________________________________________________________")
print(f"Estimated number of undamaged cars rented out from each depot and day: ")
print("_________________________________________________________________________________")

rentedOut = {}

for d in depots:
    for t in days:
        count = 0
        for d2 in depots:
            for r in rentDays:
                #print(f"Depot {d}, day {t}: cars rented out {tr[d,t].x}")
                count += pctUndamaged*pctFromToD[d,d2]*pctRent[r]*tr[d,t].x
        rentedOut[d,t] = round(count)
    

#print(rentedOut)

rentout_cars = pd.DataFrame(columns=['Day','Glasgow','Manchester','Birmingham','Plymouth'])
for t in days:
    rentout_cars = rentout_cars.append({"Day": dayname[t], "Glasgow": round(rentedOut['Glasgow',t]), "Manchester":  round(rentedOut['Manchester',t]), 'Birmingham':  round(rentedOut['Birmingham',t]),'Plymouth': round(rentedOut['Plymouth',t])   }, ignore_index=True) 
rentout_cars.index=[''] * len(rentout_cars)
rentout_cars

_________________________________________________________________________________
Estimated number of undamaged cars rented out from each depot and day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,61,89,86,37
,Tuesday,59,85,140,36
,Wednesday,63,72,111,39
,Thursday,62,103,100,38
,Friday,63,92,63,39
,Saturday,60,85,112,36


---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC